# <span style = 'color: #81A5FF'>  Exploratory Data Analysis and Cleaning </span>

### 1. Importing Libraries and data

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#from currency_converter import CurrencyConverter
import datetime
import requests
import random
import re
from sklearn.metrics.pairwise import haversine_distances
from math import radians
# from geopy.geocoders import Nominatim
from functions import *
from locations import *


%load_ext autoreload
%autoreload 2

In [ ]:
data = pd.read_csv('data/alltheforkscrapes2.csv', index_col=0)

In [ ]:
data.reset_index(inplace=True)
data.drop(['index'], axis=1, inplace=True)

In [ ]:
data

In [ ]:
data[data.duplicated()]

In [ ]:
#Drop duplicated rows
data.drop_duplicates(inplace=True, keep='last')

In [ ]:
data.reset_index(inplace=True)
data.drop(['index'], axis=1, inplace=True)

### 2. Summary Statistics

In [ ]:
data.describe().T

In [ ]:
data.describe(include='object').T

### 3. Handling Locations

In [ ]:
locations = data[['name','address', 'location', 'longitude', 'latitude']].copy()

In [ ]:
#Finds last occurence of the address to determine the city
locations['location2'] = locations['address'].map(lambda x: str(x).split(',')[-1])

#Finds the first occurence within "location" to find city -> this was how we eextracted the data
locations['location3'] = locations['location'].map(lambda x: str(x).split(',')[0])

In [ ]:
#Finding indexes from observations not based in Portugal and removing them from the original data and creating a new dataframe with only portuguese location info
foreign_indexes = list((locations[(locations['location'].isna())]).index)
data.drop(foreign_indexes, inplace=True, axis=0)
data.reset_index(inplace=True)
data.drop(['index'], axis=1, inplace=True)
pt_locations = locations[locations['location'].isna() == False].copy()
pt_locations.reset_index(inplace=True)
pt_locations.drop(['index'], axis=1, inplace=True)

In [ ]:
#Pre processing address to further find latitude and longitude coordinates
pt_locations['address'] = pt_locations['address'].apply(preprocess_address)

In [ ]:
# pt_locations[['latitude', 'longitude']] = pt_locations['address'].apply(lambda x: pd.Series(find_coordinates2(x)))

In [ ]:
# pt_locations.to_csv('portuguese_locations.csv')

In [ ]:
#Step intermédio enquanto o notebook nao estiver finalizado
pt_locations2 = pd.read_csv('data/portuguese_locations.csv', index_col=0)
pt_locations[['latitude', 'longitude']] = pt_locations2[['latitude', 'longitude']].copy()

In [ ]:
data[['address', 'latitude', 'longitude', 'location', 'city']] = pt_locations[['address', 'latitude', 'longitude', 'location2', 'location3']].copy()

### 4. Handling Missing Values and Standardizing Values

In [ ]:
null_columns = data.isnull().any()
null_columns = null_columns[null_columns].index
#list(null_columns)

#### Chef Name

In [ ]:
data['chefName'] = np.where(data['chefName'].isnull(), 'Not Applicable', data['chefName'])

In [ ]:
import re
data['chefName'] = data['chefName'].str.replace(r'[^A-Za-z\s]', '', regex=True)
data['chefName'] = data['chefName'].str.replace('Chefes', '', case=False).str.strip()
data['chefName'] = data['chefName'].str.replace('Chefe', '', case=False).str.strip()
data['chefName'] = data['chefName'].str.replace('Chef', '', case=False).str.strip()
data['chefName'] = data['chefName'].str.replace('executivos', '', case=False).str.strip()
data['chefName'] = data['chefName'].str.replace('executivo', '', case=False).str.strip()
data['chefName'] = data['chefName'].str.replace('Chefe executivo', '', case=False).str.strip()
data['chefName'] = data['chefName'].str.replace('Chef', '', case=False).str.strip()
data['chefName'] = data['chefName'].str.replace('  ', ' ', case=False).str.strip()
data['chefName'].to_csv('chefs.csv', index=False)


#### Phone Number

In [ ]:
data['phone'] = data['phone'].astype(str)
data['phone'] = data['phone'].apply(lambda x: x[3:12] if x != 'nan' else 'Not Available')
data['phone'] = np.where(data['phone'].str.endswith('.'), '966287253', data['phone'])

Aqui falta fazer description, ratingValue, maxPartySize, reviewCount

#### Dealing with Schedule Format

In [ ]:
data['openingHours'].fillna('Not Available', inplace=True)

In [ ]:
#Clearing the openingHours column
data['schedule'] = data['openingHours'].apply(lambda x: clean_openinghours(x))

#### Generating Promotions

In [ ]:
data['promotions'] = data['schedule'].apply(lambda x: promotion_generator(x, 3, 0.5))

### 6. Exploratory Data Analysis (through Sections)

#### 6.1. Photos

In [ ]:
photos = ['photo', 'customerPhotos/0','customerPhotos/1', 'customerPhotos/2', 'customerPhotos/3', 'customerPhotos/4', 'customerPhotos/5', 'customerPhotos/6',
 'customerPhotos/7', 'customerPhotos/8', 'customerPhotos/9', 'customerPhotos/10', 'customerPhotos/11', 'customerPhotos/12', 'customerPhotos/13',
 'customerPhotos/14', 'customerPhotos/15', 'customerPhotos/16', 'customerPhotos/17', 'customerPhotos/18', 'customerPhotos/19', 'photos/1',
 'photos/2', 'photos/3', 'photos/4', 'photos/5', 'photos/6', 'photos/7','photos/8','photos/9','photos/10', 'photos/11', 'photos/12', 'photos/13', 'photos/14',
 'photos/15','photos/16', 'photos/17', 'photos/18', 'photos/19', 'photos/20', 'photos/21', 'photos/22', 'photos/23', 'photos/24', 'photos/25', 'photos/26', 'photos/27',
 'photos/28', 'photos/29', 'photos/30', 'photos/31', 'photos/32', 'photos/33', 'photos/34', 'photos/35', 'photos/36', 'photos/37', 'photos/38', 'photos/39', 'photos/40',
 'photos/41', 'photos/42', 'photos/43', 'photos/44', 'photos/45', 'photos/46', 'photos/47', 'photos/48', 'photos/49', 'photos/50', 'photos/51', 'photos/52','photos/53',
 'photos/54', 'photos/55', 'photos/56', 'photos/57', 'photos/58', 'photos/59', 'photos/60', 'photos/61', 'photos/62', 'photos/63', 'photos/64', 'photos/65', 'photos/66',
 'photos/67','photos/68','photos/69', 'photos/70', 'photos/71', 'photos/72', 'photos/73', 'photos/74', 'photos/75', 'photos/76', 'photos/77', 'photos/78', 'photos/79',
 'photos/80', 'photos/81', 'photos/82', 'photos/83', 'photos/84', 'photos/85', 'photos/86', 'photos/87', 'photos/88', 'photos/89', 'photos/90',  'photos/91', 'photos/92',
 'photos/93', 'photos/94', 'photos/95', 'photos/96', 'photos/97', 'photos/98', 'photos/99', 'photos/100', 'photos/101', 'photos/102', 'photos/103', 'photos/104', 'photos/105']

In [ ]:
df_photos = data[photos].copy()

In [ ]:
df_photos['photos/1'].unique()

#### 6.2. Reviews

In [ ]:
reviews = ['reviewList/0/ambienceRatingValue','reviewList/0/date', 'reviewList/0/foodRatingValue', 'reviewList/0/ratingValue', 'reviewList/0/review',
 'reviewList/0/reviewerName', 'reviewList/0/serviceRatingValue', 'reviewList/1/ambienceRatingValue', 'reviewList/1/date', 'reviewList/1/foodRatingValue', 'reviewList/1/ratingValue',
 'reviewList/1/review', 'reviewList/1/reviewerName', 'reviewList/1/serviceRatingValue', 'reviewList/2/ambienceRatingValue', 'reviewList/2/date', 'reviewList/2/foodRatingValue',
 'reviewList/2/ratingValue', 'reviewList/2/review', 'reviewList/2/reviewerName', 'reviewList/2/serviceRatingValue', 'reviewList/3/ambienceRatingValue','reviewList/3/date',
 'reviewList/3/foodRatingValue', 'reviewList/3/ratingValue', 'reviewList/3/review', 'reviewList/3/reviewerName', 'reviewList/3/serviceRatingValue', 'reviewList/4/ambienceRatingValue',
 'reviewList/4/date','reviewList/4/foodRatingValue', 'reviewList/4/ratingValue', 'reviewList/4/review', 'reviewList/4/reviewerName', 'reviewList/4/serviceRatingValue',
 'reviewList/5/ambienceRatingValue', 'reviewList/5/date', 'reviewList/5/foodRatingValue', 'reviewList/5/ratingValue', 'reviewList/5/review', 'reviewList/5/reviewerName',
 'reviewList/5/serviceRatingValue', 'reviewList/6/ambienceRatingValue', 'reviewList/6/date', 'reviewList/6/foodRatingValue', 'reviewList/6/ratingValue', 'reviewList/6/review',
 'reviewList/6/reviewerName', 'reviewList/6/serviceRatingValue', 'reviewList/7/ambienceRatingValue', 'reviewList/7/date', 'reviewList/7/foodRatingValue', 'reviewList/7/ratingValue',
 'reviewList/7/review', 'reviewList/7/reviewerName', 'reviewList/7/serviceRatingValue', 'reviewList/8/ambienceRatingValue', 'reviewList/8/date', 'reviewList/8/foodRatingValue',
 'reviewList/8/ratingValue', 'reviewList/8/review', 'reviewList/8/reviewerName', 'reviewList/8/serviceRatingValue', 'reviewList/9/ambienceRatingValue', 'reviewList/9/date',
 'reviewList/9/foodRatingValue', 'reviewList/9/ratingValue', 'reviewList/9/review', 'reviewList/9/reviewerName', 'reviewList/9/serviceRatingValue', 'reviewList/10/ambienceRatingValue',
 'reviewList/10/date', 'reviewList/10/foodRatingValue', 'reviewList/10/ratingValue', 'reviewList/10/review', 'reviewList/10/reviewerName', 'reviewList/10/serviceRatingValue',
 'reviewList/11/ambienceRatingValue', 'reviewList/11/date', 'reviewList/11/foodRatingValue', 'reviewList/11/ratingValue', 'reviewList/11/review', 'reviewList/11/reviewerName',
 'reviewList/11/serviceRatingValue', 'reviewList/12/ambienceRatingValue', 'reviewList/12/date', 'reviewList/12/foodRatingValue', 'reviewList/12/ratingValue', 'reviewList/12/review',
 'reviewList/12/reviewerName', 'reviewList/12/serviceRatingValue', 'reviewList/13/ambienceRatingValue', 'reviewList/13/date', 'reviewList/13/foodRatingValue', 'reviewList/13/ratingValue',
 'reviewList/13/review', 'reviewList/13/reviewerName', 'reviewList/13/serviceRatingValue', 'reviewList/14/ambienceRatingValue', 'reviewList/14/date',
 'reviewList/14/foodRatingValue', 'reviewList/14/ratingValue', 'reviewList/14/review', 'reviewList/14/reviewerName', 'reviewList/14/serviceRatingValue', 'reviewList/15/ambienceRatingValue',
 'reviewList/15/date', 'reviewList/15/foodRatingValue', 'reviewList/15/ratingValue', 'reviewList/15/review', 'reviewList/15/reviewerName', 'reviewList/15/serviceRatingValue',
 'reviewList/16/ambienceRatingValue', 'reviewList/16/date', 'reviewList/16/foodRatingValue', 'reviewList/16/ratingValue', 'reviewList/16/review', 'reviewList/16/reviewerName',
 'reviewList/16/serviceRatingValue', 'reviewList/17/ambienceRatingValue', 'reviewList/17/date', 'reviewList/17/foodRatingValue', 'reviewList/17/ratingValue', 'reviewList/17/review',
 'reviewList/17/reviewerName', 'reviewList/17/serviceRatingValue', 'reviewList/18/ambienceRatingValue', 'reviewList/18/date', 'reviewList/18/foodRatingValue', 'reviewList/18/ratingValue',
 'reviewList/18/review', 'reviewList/18/reviewerName', 'reviewList/18/serviceRatingValue', 'reviewList/19/ambienceRatingValue', 'reviewList/19/date', 'reviewList/19/foodRatingValue',
 'reviewList/19/ratingValue', 'reviewList/19/review', 'reviewList/19/reviewerName', 'reviewList/19/serviceRatingValue']

In [ ]:
df_reviews = data[reviews].copy()

In [ ]:
ambience = []
food = []
service = []
for col in df_reviews.columns:
    if 'ambienceRatingValue' in col:
        ambience.append(col)
    elif 'foodRatingValue' in col:
        food.append(col)
    elif 'serviceRatingValue' in col:
        service.append(col)

In [ ]:
#Creating three new columns regarding ratings per category using the mean values of the user-based ratings
df_reviews['ambienceRatingSummary'] = df_reviews[ambience].mean(axis=1)
df_reviews['foodRatingSummary'] = df_reviews[food].mean(axis=1)
df_reviews['serviceRatingSummary'] = df_reviews[service].mean(axis=1)

In [ ]:
data[['ambienceRatingSummary', 'foodRatingSummary', 'serviceRatingSummary']] = df_reviews[['ambienceRatingSummary', 'foodRatingSummary', 'serviceRatingSummary']].copy()

#### 6.3. Payment Methods

In [23]:
payments = ['paymentAccepted/0','paymentAccepted/1', 'paymentAccepted/2', 'paymentAccepted/3', 'paymentAccepted/4',
 'paymentAccepted/5', 'paymentAccepted/6', 'paymentAccepted/7', 'paymentAccepted/8', 'paymentAccepted/9', 'paymentAccepted/10']

In [24]:
df_payments = data[payments].copy()

In [25]:
df_payments.fillna(0, inplace=True)

In [26]:
df_payments['paymentAcceptedSummary'] = df_payments.apply(lambda row: [row[col] for col in df_payments.columns if row[col] != 0], axis=1)

In [27]:
data['paymentAcceptedSummary'] = df_payments['paymentAcceptedSummary'].copy()

In [93]:
data_exploded_pay = data['paymentAcceptedSummary'].explode()
payment_counts = data_exploded_pay.value_counts()

print(payment_counts)

paymentAcceptedSummary
Credit Card         1345
Visa                1197
Mastercard          1049
Visa Electron        560
MBWay                415
Maestro Card         274
American Express     212
Apple Pay             60
Cash                  39
Diners Club Card      24
Voucher               21
Check                  8
Bancontact             5
Sodexo                 2
Name: count, dtype: int64


In [92]:
#Replace 'Cash Only' with 'Cash'
data['paymentAcceptedSummary'] = [x if 'Cash Only' not in x else [item.replace('Cash Only', 'Cash') for item in x] for x in data['paymentAcceptedSummary']]

#Replace 'Elo', 'Cabal Credit Card', 'Cabal', 'MobilePay', 'Rede Shop', 'Clave', 'EC Card' with 'MBWay'
data['paymentAcceptedSummary'] = [ [item if item not in ['Elo', 'Cabal Credit Card', 'Cabal Debit Card', 'MobilePay', 'Rede Shop',
                                                         'Clave Debit Card', 'Clave Credit Card', 'EC card'] else 'MBWay' for item in x]
                                  for x in data['paymentAcceptedSummary']]

# Eleminate duplicates from the list
data['paymentAcceptedSummary'] = [list(set(x)) for x in data['paymentAcceptedSummary']]

#### 6.4. Tags

In [ ]:
df_tags = data[['tags/0','tags/1', 'tags/2', 'tags/3', 'tags/4', 'tags/5', 'tags/6', 'tags/7',]].copy()
df_tags

In [ ]:
# df_tags['tags/0'].unique()

In [ ]:
# df_tags['tags/1'].unique()

In [ ]:
# df_tags['tags/2'].unique()

In [ ]:
# df_tags['tags/3'].unique()

In [ ]:
# df_tags['tags/4'].unique()

In [ ]:
# df_tags['tags/5'].unique()

In [ ]:
# df_tags['tags/6'].unique()

In [ ]:
# df_tags['tags/7'].unique()

In [ ]:
# df_tags.to_csv('tags.csv', index = True)

In [ ]:
df_tags['Michelin'] = df_tags.apply(lambda row: 1 if 'MICHELIN' in row.values else 0, axis=1)

In [ ]:
df_tags[df_tags['Michelin']==1]

In [ ]:
#Removing TheFork based values
df_tags = df_tags.replace('MICHELIN', np.nan)
df_tags = df_tags.replace(['YUMS x2'], np.nan)
df_tags = df_tags.replace(['Accepting my yums'], np.nan)
df_tags = df_tags.replace(['INSIDER'], np.nan)

In [ ]:
#unique values in the df_tags dataframe
for col in df_tags.columns:
    print(df_tags[col].unique())

In [ ]:
df_tags

In [ ]:
df_tags.info()

In [ ]:
# remove the last 5 columns from the dataframe
df_tags.drop(df_tags.columns[-6:], axis=1, inplace=True)

In [ ]:
for col in df_tags.columns:
    print(df_tags[col].unique())

In [ ]:
# join these columns in one column just with the non null values
df_tags['type'] = df_tags.apply(lambda row: next((val for val in row if not pd.isna(val)), np.nan), axis=1)


In [ ]:
# delete all the columns except the new one
df_tags.drop(df_tags.columns[:-1], axis=1, inplace=True)

In [ ]:
# CREATE A CSV FILE WITH DF_TAGS DATAFRAME WITH NAME TYPES
df_tags.to_csv('types.csv', index = True)

#### General Metrics

In [ ]:
df_workable = data[['address', 'averagePrice', 'chefName', 'cuisine', 'currency', 'description', 'hasLoyaltyProgram', 'isBookable', 'maxPartySize',
 'name', 'offer', 'schedule', 'promotions', 'phone', 'photo', 'ratingValue', 'reviewCount', 'style', 'latitude', 'longitude', 'location', 'city',
 'ambienceRatingSummary', 'foodRatingSummary', 'serviceRatingSummary', 'paymentAcceptedSummary', ]].copy()

In [ ]:
# #If not euros, convert the average price to euros.
# c = CurrencyConverter()
# df_workable['averagePriceEUR'] = df_workable.apply(to_euros, axis=1)

In [ ]:
df_workable.isnull().any()

### Visualizing Data

In [ ]:
df_workable['style'].unique()

In [ ]:
import folium

m = folium.Map(location=(38.736946, -9.142685), zoom_start=7, tiles="cartodb positron")


### Restaurants Near You

In [ ]:
df1 = df_workable[df_workable['location'] == 'Lisbon']
shaul = Location()
shaul.getLocation()

In [ ]:
df1 = nearYou(shaul, df1, 10)

In [ ]:
df1['minutes_away'] = df1.apply(lambda row: shaul.getDirections(row['latitude'], row['longitude'], ['driving'])['driving'].minutes, axis=1)

In [ ]:
df1

__Painel de Analytics (primeira página)__:
- dizer quantos restaurantes se encontraram perto (Fazer um plot);
- nome do restaurante mais próximo e distancia;
- qual o tipo de comida mais comum próximo;
- Qual o average price da comdia proximo;
- Qual o restaurante mais top rated e mesmo a nível de coisas especificas (ambiente, comida);
- Promoções perto.
- User poderá escolher a data para ver os restaurantes abertps;

In [ ]:
import folium

m = folium.Map(location=(shaul.latitude, shaul.longitude), zoom_start=7, tiles="cartodb positron")
for index, row in df1.iterrows():
    # Create a marker for each observation
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['name'],  # Display the name in a popup
    ).add_to(m)

m

In [ ]:
df1['averagePrice'].median()

In [ ]:
df1['cuisine'].value_counts()

In [ ]:
df1[df1['minutes_away'] == df1['minutes_away'].min()]

In [ ]:
df1[df1['ratingValue'] == df1['ratingValue'].max()]

In [ ]:
df1[df1['foodRatingSummary'] == df1['foodRatingSummary'].max()]

In [ ]:
df1[df1['ambienceRatingSummary'] == df1['ambienceRatingSummary'].max()]

In [ ]:
df1[df1['serviceRatingSummary'] == df1['serviceRatingSummary'].max()]

In [ ]:
df1[df1['promotions'] != 'No Offers']